In [1]:
import bpy
import numpy as np
from pickle import load
from arm_movement import max_left_shoulder_rotation, max_right_shoulder_rotation, landmarks2arm_position, arm_position2landmarks
from config import pi
from mathutils import Vector, Matrix

In [2]:
with open("../Datos/Procesados/alphabet_landmarks_spread.pkl", "rb") as f:
    total_results = load(f)

In [3]:
def get_half_landmarks(data, letter, frame):
    half_landmarks = np.empty((48, 3), dtype=np.float64)
    half_landmarks[:6] = data[letter]["landmarks"]["pose"][frame, [23, 24] + list(range(11, 15)), :-1]
    half_landmarks[6::2] = data[letter]["landmarks"]["left_hand"][frame]
    half_landmarks[6::2, -1] += data[letter]["landmarks"]["pose"][frame, 15, 2]
    half_landmarks[7::2] = data[letter]["landmarks"]["right_hand"][frame]
    half_landmarks[7::2, -1] += data[letter]["landmarks"]["pose"][frame, 16, 2]
    return half_landmarks

In [4]:
def get_joints(half_landmarks, scale=np.array([1, 1, 1], dtype=np.uint8)):
    joints = np.empty((51, 3), dtype=np.float64)
    joints[[1, 2] + list(range(5, 51))] = (half_landmarks * scale)[:, [0, 2, 1]] * np.array([1, 1, -1])
    joints[0] = joints[[1, 2]].mean(axis=0)
    joints[4] = joints[[5, 6]].mean(axis=0)
    joints[3] = joints[[0, 4]].mean(axis=0)
    return joints

In [5]:
bone_conexions_list = [[0, 1], [0, 2], [0, 3], [3, 4], [4, 5], [4, 6]]
bone_names = ["l_hip", "r_hip", "d_backbone", "u_backbone", "l_shoulder", "r_shoulder"]
bone_parents_list = [-1, -1, -1, 2, 3, 3]

bone_conexions_list.extend([[i, i+2] for i in range(5, 9)])
bone_names.extend(["%s_%s" % (side, part) for part in ["upperarm", "forearm"] for side in ["l", "r"]])
bone_parents_list.extend(range(4, 8))

bone_conexions_list.extend([[9+side+2*(i!=0)*(4*finger+i), 9+side+2*(4*finger+i+1)] for finger in range(5) for i in range(4) for side in range(2)])
bone_names.extend(["%s_%s_%d" % (side, finger_name, i)
                   for finger_name in ["thumb", "index", "middle", "ring", "pinky"] for i in range(4) for side in ["l", "r"]])
bone_parents_list.extend([8+side+2*(i!=0)*(4*finger+i) for finger in range(5) for i in range(4) for side in range(2)])

bone_conexions = np.array(bone_conexions_list, dtype=np.uint8)
bone_parents = np.array(bone_parents_list, dtype=np.int8)

In [6]:
def generate_armature(joints, bone_conexions, bone_parents, bone_names, filepath):
    """
    Given an array with the coordinates of the joints, an array of shape (number of bones, 2)
    specifying the head and tail of each bone, an array of shape (number of bones,) specifying
    the parent of each bone, a list of strings and a filepath, generates a Blender armature.

    Inputs:
      - joints (array of shape (number of joints, 3)): each row specify the coordinates of a joint.
      - bone_conexions (array of shape (number of bones, 2)): each row specify the head and tail of
      a bone by giving the location of both joints in the array of joints.
      - bone_parents (array of shape (number of bones,)): each element specify the parent of the
      corresponding bone by giving the location of its parent.
      - bone_names (list of str): each element is the name of the corresponding bone.
      - filepath (str): the path to the file where the armature is being saved.
    """
    bpy.ops.wm.read_factory_settings(use_empty=True)

    bpy.ops.object.armature_add(location=(0, 0, 0))
    armature = bpy.context.object

    bpy.ops.object.mode_set(mode='EDIT')

    bone_number = 0

    bone = armature.data.edit_bones[bone_number]
    bone.head, bone.tail = joints[bone_conexions[bone_number]]
    bone.name = bone_names[bone_number]

    for conexion in bone_conexions[1:]:
        bone_number += 1
        bone_name = bone_names[bone_number]
        bone = armature.data.edit_bones.new(bone_name)

        bone.head, bone.tail = joints[bone_conexions[bone_number]]

        if bone_parents[bone_number]>=0:
            bone.parent = armature.data.edit_bones[bone_names[bone_parents[bone_number]]]
            bone.use_connect = True

    bpy.ops.object.mode_set(mode='OBJECT')

    bpy.ops.wm.save_as_mainfile(filepath=filepath)

In [7]:
trunk_landmarks_avatar = np.array([
    [0.15, 0., 0.],
    [-0.15, 0., 0.],
    [0.2, -0.5, 0.],
    [-0.2, -0.5, 0.]
], dtype=np.float64)

left_finger_directions = np.array([
    [[0, -np.sin(0.31*pi), np.cos(0.31*pi)],
     [0, -np.sin(0.09*pi), np.cos(0.09*pi)],
     [0, -np.sin(0.02*pi), np.cos(0.02*pi)],
     [0, np.sin(0.03*pi), np.cos(0.03*pi)],
     [0, np.sin(0.11*pi), np.cos(0.11*pi)]],
    [[0, -np.sin(0.17*pi), np.cos(0.17*pi)],
     [0, 0, 1],
     [0, 0, 1],
     [0, 0, 1],
     [0, 0, 1]],
    [[0, -np.sin(0.17*pi), np.cos(0.17*pi)],
     [0, 0, 1],
     [0, 0, 1],
     [0, 0, 1],
     [0, 0, 1]],
    [[0, -np.sin(0.17*pi), np.cos(0.17*pi)],
     [0, 0, 1],
     [0, 0, 1],
     [0, 0, 1],
     [0, 0, 1]]
], dtype=np.float64)
right_finger_directions = np.array([
    [[0, np.sin(0.31*pi), np.cos(0.31*pi)],
     [0, np.sin(0.09*pi), np.cos(0.09*pi)],
     [0, np.sin(0.02*pi), np.cos(0.02*pi)],
     [0, -np.sin(0.03*pi), np.cos(0.03*pi)],
     [0, -np.sin(0.11*pi), np.cos(0.11*pi)]],
    [[0, np.sin(0.17*pi), np.cos(0.17*pi)],
     [0, 0, 1],
     [0, 0, 1],
     [0, 0, 1],
     [0, 0, 1]],
    [[0, np.sin(0.17*pi), np.cos(0.17*pi)],
     [0, 0, 1],
     [0, 0, 1],
     [0, 0, 1],
     [0, 0, 1]],
    [[0, np.sin(0.17*pi), np.cos(0.17*pi)],
     [0, 0, 1],
     [0, 0, 1],
     [0, 0, 1],
     [0, 0, 1]]
], dtype=np.float64)
left_finger_lengths = np.array([
    [0.03, 0.103, 0.103, 0.1, 0.09],
    [0.045, 0.031, 0.035, 0.032, 0.025],
    [0.032, 0.024, 0.026, 0.023, 0.017],
    [0.033, 0.024, 0.025, 0.025, 0.023]
], dtype=np.float64).reshape(4, 5, 1)
right_finger_lengths = left_finger_lengths.copy()

arms_position_rest = {
    "left_shoulder_direction": np.array([0, 1, 0], dtype=np.float64),
    "left_upperarm_length": np.float64(0.33),
    "left_shoulder_rotation": pi/2.,
    "left_elbow_angle": pi,
    "left_forearm_length": np.float64(0.27),
    "left_elbow_rotation": np.float64(0),
    "left_wrist_rotation": np.float64(0),
    "left_wrist_inclination": np.float64(0),
    "left_finger_directions": left_finger_directions,
    "left_finger_lengths": left_finger_lengths,

    "right_shoulder_direction": np.array([0, 1, 0], dtype=np.float64),
    "right_upperarm_length": np.float64(0.33),
    "right_shoulder_rotation": pi/2.,
    "right_elbow_angle": pi,
    "right_forearm_length": np.float64(0.27),
    "right_elbow_rotation": np.float64(0),
    "right_wrist_rotation": np.float64(0),
    "right_wrist_inclination": np.float64(0),
    "right_finger_directions": right_finger_directions,
    "right_finger_lengths": right_finger_lengths
}

half_landmarks_rest = arm_position2landmarks(arms_position_rest, trunk_landmarks_avatar, 1, 1)

In [28]:
generate_armature(get_joints(half_landmarks_rest), bone_conexions, bone_parents, bone_names, "../Datos/Procesados/avatar_armature.blend")

Info: Saved "avatar_armature.blend"


In [8]:
def move_arms_blender_v1(
    armature_filepath, armature_name, half_landmarks_armature, arms_position_armature,
    list_half_landmarks, list_frame_dimensions, list_time_movement, list_time_position,
    fps, animation_filepath
):
    # Obtain the armature
    bpy.ops.wm.open_mainfile(filepath=armature_filepath)
    armature = bpy.data.objects.get(armature_name)
    bpy.context.view_layer.objects.active = armature
    arms_bones_names = bone_names[6:]

    def set_pose(half_world_landmarks_0, arms_position_0, arms_position_1):
        half_world_landmarks_ = half_world_landmarks_0.copy()
        arms_position_ = arms_position_0.copy()

        left_shoulder_basis = np.empty((3, 3), dtype=np.float64)
        left_shoulder_basis[:, 0] = half_world_landmarks_[2] - half_world_landmarks_[3]
        left_shoulder_basis[:, 2] = np.cross(left_shoulder_basis[:, 0], half_world_landmarks_[0] - half_world_landmarks_[2])
        left_shoulder_basis[:, 1] = np.cross(left_shoulder_basis[:, 2], left_shoulder_basis[:, 0])
        left_shoulder_basis /= np.linalg.norm(left_shoulder_basis, axis=0, keepdims=True)

        right_shoulder_basis = np.empty((3, 3), dtype=np.float64)
        right_shoulder_basis[:, 0] = half_world_landmarks_[3] - half_world_landmarks_[2]
        right_shoulder_basis[:, 2] = np.cross(right_shoulder_basis[:, 0], half_world_landmarks_[1] - half_world_landmarks_[3])
        right_shoulder_basis[:, 1] = np.cross(right_shoulder_basis[:, 2], right_shoulder_basis[:, 0])
        right_shoulder_basis /= np.linalg.norm(right_shoulder_basis, axis=0, keepdims=True)


        # Upperarm direction and rotation
        bone_name = "l_upperarm"
        bone = armature.pose.bones[bone_name]
        bone.rotation_mode = 'YXZ'

        left_upperarm_image = arms_position_1["left_shoulder_direction"] @ left_shoulder_basis.T
        v1, v2, v3 = bone.matrix.transposed().to_3x3() @ Vector(left_upperarm_image[[0, 2, 1]] * np.array([1, 1, -1]))

        if np.isclose(v2, 1):
            alpha = arms_position_1["left_shoulder_rotation"] - arms_position_["left_shoulder_rotation"]
            beta = 0.
            gamma = 0.
        else:
            if np.isclose(v3*v3, 1):
                beta = np.sign(v3) * pi/2.
                gamma = 0.
            else:
                gamma = np.arctan2(-v1, v2)
                beta = np.arctan2(v3, -v1/np.sin(gamma)) if np.isclose(v2, 0) else np.arctan2(v3, v2/np.cos(gamma))

            rotation_matrix = np.array([
                [np.cos(gamma), np.sin(gamma), 0],
                [- np.cos(beta) * np.sin(gamma), np.cos(beta) * np.cos(gamma), np.sin(beta)],
                [np.sin(beta) * np.sin(gamma), - np.sin(beta) * np.cos(gamma), np.cos(beta)]
            ], dtype=np.float64)
            bone_matrix_np = np.array(bone.matrix.to_3x3(), dtype=np.float64)
            left_arm_vectors = (half_world_landmarks_[2*np.arange(2, 24)] - half_world_landmarks_[2])[:, [0, 2, 1]] * np.array([1, 1, -1])

            half_world_landmarks_[2*np.arange(2, 24)] = ((left_arm_vectors @ bone_matrix_np @ rotation_matrix @ bone_matrix_np.T)[:, [0, 2, 1]] *
                                                        np.array([1, -1, 1]) + half_world_landmarks_[2])
            if np.isclose(arms_position_["left_elbow_angle"], pi):
                aux_position = landmarks2arm_position(half_world_landmarks_, 1, 1)
                left_shoulder_rotation = (aux_position["left_shoulder_rotation"] +
                                          aux_position["left_elbow_rotation"] - arms_position_["left_elbow_rotation"])
                alpha = arms_position_1["left_shoulder_rotation"] - left_shoulder_rotation
            else:
                alpha = arms_position_1["left_shoulder_rotation"] - landmarks2arm_position(half_world_landmarks_, 1, 1)["left_shoulder_rotation"]
        bone.rotation_euler = (beta, alpha, gamma)

        bone_name = "r_upperarm"
        bone = armature.pose.bones[bone_name]
        bone.rotation_mode = 'YXZ'

        right_upperarm_image = arms_position_1["right_shoulder_direction"] @ right_shoulder_basis.T
        v1, v2, v3 = bone.matrix.inverted().to_3x3() @ Vector(right_upperarm_image[[0, 2, 1]] * np.array([1, 1, -1]))

        if np.isclose(v2, 1):
            alpha = arms_position_["right_shoulder_rotation"] - arms_position_1["right_shoulder_rotation"]
            beta = 0.
            gamma = 0.
        else:
            if np.isclose(v3*v3, 1):
                beta = np.sign(v3) * pi/2.
                gamma = 0.
            else:
                gamma = np.arctan2(-v1, v2)
                beta = np.arctan2(v3, -v1/np.sin(gamma)) if np.isclose(v2, 0) else np.arctan2(v3, v2/np.cos(gamma))

            rotation_matrix = np.array([
                [np.cos(gamma), np.sin(gamma), 0],
                [- np.cos(beta) * np.sin(gamma), np.cos(beta) * np.cos(gamma), np.sin(beta)],
                [np.sin(beta) * np.sin(gamma), - np.sin(beta) * np.cos(gamma), np.cos(beta)]
            ], dtype=np.float64)
            bone_matrix_np = np.array(bone.matrix.to_3x3(), dtype=np.float64)
            right_arm_vectors = (half_world_landmarks_[2*np.arange(2, 24)+1] - half_world_landmarks_[3])[:, [0, 2, 1]] * np.array([1, 1, -1])

            half_world_landmarks_[2*np.arange(2, 24)+1] = ((right_arm_vectors @ bone_matrix_np @ rotation_matrix @ bone_matrix_np.T)[:, [0, 2, 1]] *
                                                           np.array([1, -1, 1]) + half_world_landmarks_[3])

            if np.isclose(arms_position_["right_elbow_angle"], pi):
                aux_position = landmarks2arm_position(half_world_landmarks_, 1, 1)
                right_shoulder_rotation = (aux_position["right_shoulder_rotation"] +
                                          aux_position["right_elbow_rotation"] - arms_position_["right_elbow_rotation"])
                alpha = right_shoulder_rotation - arms_position_1["right_shoulder_rotation"]
            else:
                alpha = landmarks2arm_position(half_world_landmarks_, 1, 1)["right_shoulder_rotation"] - arms_position_1["right_shoulder_rotation"]
        bone.rotation_euler = (beta, alpha, gamma)

        # Update position
        bpy.context.view_layer.update()
        for bone_number in range(6, 50):
            bone = armature.pose.bones[bone_names[bone_number]]
            half_world_landmarks_[bone_number-2] = np.array(bone.tail, dtype=np.float64)[[0, 2, 1]] * np.array([1, -1, 1])

        left_arm_is_extended = np.isclose(arms_position_["left_elbow_angle"], pi)
        right_arm_is_extended = np.isclose(arms_position_["right_elbow_angle"], pi)
        left_elbow_rotation = arms_position_["left_elbow_rotation"]
        right_elbow_rotation = arms_position_["right_elbow_rotation"]
        arms_position_.update(landmarks2arm_position(half_world_landmarks_, 1, 1))
        if left_arm_is_extended:
            arms_position_["left_shoulder_rotation"] = (arms_position_["left_shoulder_rotation"] +
                                                        arms_position_["left_elbow_rotation"] - left_elbow_rotation)
            arms_position_["left_elbow_rotation"] = left_elbow_rotation
        if right_arm_is_extended:
            arms_position_["right_shoulder_rotation"] = (arms_position_["right_shoulder_rotation"] +
                                                         arms_position_["right_elbow_rotation"] - right_elbow_rotation)
            arms_position_["right_elbow_rotation"] = right_elbow_rotation


        # Forearm direction and rotation
        bone_name = "l_forearm"
        bone = armature.pose.bones[bone_name]
        bone.rotation_mode = 'YXZ'

        if np.isclose(arms_position_["left_elbow_angle"], arms_position_1["left_elbow_angle"]):
            alpha = arms_position_1["left_elbow_rotation"] - arms_position_["left_elbow_rotation"]
            beta = 0.
            gamma = 0.
        else:
            bone_matrix = bone.matrix.to_3x3()

            v2 = (arms_position_["left_shoulder_direction"] @ left_shoulder_basis.T)[[0, 2, 1]] * np.array([1, 1, -1])
            if np.isclose(arms_position_["left_elbow_angle"], pi):
                left_arm_plane_normal_vector_max_rotation = (max_left_shoulder_rotation(arms_position_["left_shoulder_direction"]) @
                                                             left_shoulder_basis.T)[[0, 2, 1]] * np.array([1, 1, -1])
                aux_vector = np.cross(v2, left_arm_plane_normal_vector_max_rotation)
                left_arm_plane_normal_vector = (np.cos(arms_position_["left_shoulder_rotation"]) * left_arm_plane_normal_vector_max_rotation +
                                               np.sin(arms_position_["left_shoulder_rotation"]) * aux_vector)
                v1 = - left_arm_plane_normal_vector
            else:
                left_forearm_direction = (half_world_landmarks_[6] - half_world_landmarks_[4])[[0, 2, 1]] * np.array([1, 1, -1])
                left_forearm_direction /= np.linalg.norm(left_forearm_direction)
                v1 = np.cross(left_forearm_direction, v2)
                v1 /= np.linalg.norm(v1)
            v3 = np.cross(v1, v2)
            parent_matrix = Matrix(np.array([v1, v2, v3]).T)
            # parent_matrix = bone.parent.matrix.to_3x3()

            beta_parent = arms_position_1["left_elbow_angle"] - arms_position_["left_elbow_angle"]
            rotation_matrix_parent = Matrix([
                [1, 0, 0],
                [0, np.cos(beta_parent), - np.sin(beta_parent)],
                [0, np.sin(beta_parent), np.cos(beta_parent)]
            ])
            rotation_matrix_bone = bone_matrix.transposed() @ parent_matrix @ rotation_matrix_parent @ parent_matrix.transposed() @ bone_matrix
            beta, _, gamma = rotation_matrix_bone.to_euler("YXZ")

            rotation_matrix = np.array([
                [np.cos(gamma), np.sin(gamma), 0],
                [- np.cos(beta) * np.sin(gamma), np.cos(beta) * np.cos(gamma), np.sin(beta)],
                [np.sin(beta) * np.sin(gamma), - np.sin(beta) * np.cos(gamma), np.cos(beta)]
            ], dtype=np.float64)
            bone_matrix_np = np.array(bone_matrix, dtype=np.float64)
            left_hand_vectors = (half_world_landmarks_[2*np.arange(3, 24)] - half_world_landmarks_[4])[:, [0, 2, 1]] * np.array([1, 1, -1])

            half_world_landmarks_[2*np.arange(3, 24)] = ((left_hand_vectors @ bone_matrix_np @ rotation_matrix @ bone_matrix_np.T)[:, [0, 2, 1]] *
                                                        np.array([1, -1, 1]) + half_world_landmarks_[4])

            alpha = arms_position_1["left_elbow_rotation"] - landmarks2arm_position(half_world_landmarks_, 1, 1)["left_elbow_rotation"]
        bone.rotation_euler = (beta, alpha, gamma)

        bone_name = "r_forearm"
        bone = armature.pose.bones[bone_name]
        bone.rotation_mode = 'YXZ'

        if np.isclose(arms_position_["right_elbow_angle"], arms_position_1["right_elbow_angle"]):
            alpha = arms_position_["right_elbow_rotation"] - arms_position_1["right_elbow_rotation"]
            beta = 0.
            gamma = 0.
        else:
            bone_matrix = bone.matrix.to_3x3()
    
            v2 = (arms_position_["right_shoulder_direction"] @ right_shoulder_basis.T)[[0, 2, 1]] * np.array([1, 1, -1])
            if np.isclose(arms_position_["right_elbow_angle"], pi):
                right_arm_plane_normal_vector_max_rotation = (max_right_shoulder_rotation(arms_position_["right_shoulder_direction"]) @
                                                              right_shoulder_basis.T)[[0, 2, 1]] * np.array([1, 1, -1])
                aux_vector = np.cross(right_arm_plane_normal_vector_max_rotation, v2)
                right_arm_plane_normal_vector = (np.cos(arms_position_["right_shoulder_rotation"]) * right_arm_plane_normal_vector_max_rotation +
                                                 np.sin(arms_position_["right_shoulder_rotation"]) * aux_vector)
                v1 = - right_arm_plane_normal_vector
            else:
                right_forearm_direction = (half_world_landmarks_[7] - half_world_landmarks_[5])[[0, 2, 1]] * np.array([1, 1, -1])
                right_forearm_direction /= np.linalg.norm(right_forearm_direction)
                v1 = np.cross(right_forearm_direction, v2)
                v1 /= np.linalg.norm(v1)
            v3 = np.cross(v1, v2)
            parent_matrix = Matrix(np.array([v1, v2, v3]).T)
            # parent_matrix = bone.parent.matrix.to_3x3()

            beta_parent = arms_position_1["right_elbow_angle"] - arms_position_["right_elbow_angle"]
            rotation_matrix_parent = Matrix([
                [1, 0, 0],
                [0, np.cos(beta_parent), - np.sin(beta_parent)],
                [0, np.sin(beta_parent), np.cos(beta_parent)]
            ])
            rotation_matrix_bone = bone_matrix.transposed() @ parent_matrix @ rotation_matrix_parent @ parent_matrix.transposed() @ bone_matrix
            beta, _, gamma = rotation_matrix_bone.to_euler("YXZ")

            rotation_matrix = np.array([
                [np.cos(gamma), np.sin(gamma), 0],
                [- np.cos(beta) * np.sin(gamma), np.cos(beta) * np.cos(gamma), np.sin(beta)],
                [np.sin(beta) * np.sin(gamma), - np.sin(beta) * np.cos(gamma), np.cos(beta)]
            ], dtype=np.float64)
            bone_matrix_np = np.array(bone_matrix, dtype=np.float64)
            right_hand_vectors = (half_world_landmarks_[2*np.arange(3, 24)+1] - half_world_landmarks_[5])[:, [0, 2, 1]] * np.array([1, 1, -1])

            half_world_landmarks_[2*np.arange(3, 24)+1] = ((right_hand_vectors @ bone_matrix_np @ rotation_matrix @ bone_matrix_np.T)[:, [0, 2, 1]] *
                                                        np.array([1, -1, 1]) + half_world_landmarks_[5])

            alpha = landmarks2arm_position(half_world_landmarks_, 1, 1)["right_elbow_rotation"] - arms_position_1["right_elbow_rotation"]
        bone.rotation_euler = (beta, alpha, gamma)

        # Update position
        bpy.context.view_layer.update()
        for bone_number in range(6, 50):
            bone = armature.pose.bones[bone_names[bone_number]]
            half_world_landmarks_[bone_number-2] = np.array(bone.tail, dtype=np.float64)[[0, 2, 1]] * np.array([1, -1, 1])
        arms_position_.update(landmarks2arm_position(half_world_landmarks_, 1, 1))


        # Wrist rotation and inclination, thumb direction
        bone_names_ = ["l_index_0", "l_middle_0", "l_ring_0", "l_pinky_0"]

        left_forearm_direction = half_world_landmarks_[6] - half_world_landmarks_[4]
        left_forearm_direction /= np.linalg.norm(left_forearm_direction)
        if np.isclose(arms_position_["left_elbow_angle"], pi):
            left_arm_rotation = arms_position_["left_shoulder_rotation"] + arms_position_["left_elbow_rotation"] + pi/2.
            aux_vector = max_left_shoulder_rotation(arms_position_["left_shoulder_direction"]) @ left_shoulder_basis.T
            left_palm_normal_vector_max_arm_rotation = np.cross(aux_vector, left_forearm_direction)
            left_palm_normal_vector_no_wrist_inclination = (np.cos(left_arm_rotation) * left_palm_normal_vector_max_arm_rotation +
                                                            np.sin(left_arm_rotation) * aux_vector)
        else:
            left_palm_normal_vector_no_elbow_rotation = np.cross(arms_position_["left_shoulder_direction"] @ left_shoulder_basis.T, left_forearm_direction)
            left_palm_normal_vector_no_elbow_rotation /= np.linalg.norm(left_palm_normal_vector_no_elbow_rotation)
            aux_vector = np.cross(left_forearm_direction, left_palm_normal_vector_no_elbow_rotation)
            left_palm_normal_vector_no_wrist_inclination = (np.cos(arms_position_["left_elbow_rotation"]) * left_palm_normal_vector_no_elbow_rotation +
                                                            np.sin(arms_position_["left_elbow_rotation"]) * aux_vector)
        v1 = left_palm_normal_vector_no_wrist_inclination[[0, 2, 1]] * np.array([1, 1, -1])
        v3 = left_forearm_direction[[0, 2, 1]] * np.array([1, 1, -1])
        v2 = np.cross(v3, v1)
        parent_matrix = Matrix(np.array([v1, v3, -v2]).T)
        # Si el esqueleto está totalmente ajustado, creo que se puede sustituir (igual hay que cambiar el signo del ángulo de la rotación)
        # parent_matrix = armature.pose.bones["l_forearm"].matrix.to_3x3()

        a = np.cos(arms_position_["left_wrist_inclination"])
        b = np.sin(arms_position_["left_wrist_inclination"])
        c = np.cos(arms_position_["left_wrist_rotation"])
        d = np.sin(arms_position_["left_wrist_rotation"])
        rotation_matrix_parent_0 = Matrix([
            [1.+(a-1.)*c*c, -b*c, (1.-a)*c*d],
            [b*c, a, -b*d],
            [(1.-a)*c*d, b*d, 1.+(a-1.)*d*d]
        ])
        a = np.cos(arms_position_1["left_wrist_inclination"])
        b = np.sin(arms_position_1["left_wrist_inclination"])
        c = np.cos(arms_position_1["left_wrist_rotation"])
        d = np.sin(arms_position_1["left_wrist_rotation"])
        rotation_matrix_parent_1 = Matrix([
            [1.+(a-1.)*c*c, b*c, (1.-a)*c*d],
            [-b*c, a, b*d],
            [(1.-a)*c*d, -b*d, 1.+(a-1.)*d*d]
        ])
        rotation_matrix = parent_matrix @ rotation_matrix_parent_1 @ rotation_matrix_parent_0 @ parent_matrix.transposed()
        for bone_name in bone_names_:
            bone = armature.pose.bones[bone_name]
            bone.rotation_mode = 'YXZ'

            bone_matrix = bone.matrix.to_3x3()
            rotation_matrix_bone = bone_matrix.transposed() @ rotation_matrix @ bone_matrix
            bone.rotation_euler = rotation_matrix_bone.to_euler("YXZ")

        bone_name = "l_thumb_0"
        bone = armature.pose.bones[bone_name]
        bone.rotation_mode = 'YXZ'
        bone_matrix = bone.matrix.to_3x3()

        left_hand_basis = np.empty((3, 3), dtype=np.float64)
        left_hand_basis[:, 0] = np.cross(half_world_landmarks_[40] - half_world_landmarks_[6], half_world_landmarks_[16] - half_world_landmarks_[6])
        left_hand_basis[:, 2] = (half_world_landmarks_[40] + half_world_landmarks_[16])/2. - half_world_landmarks_[6]
        left_hand_basis[:, 1] = np.cross(left_hand_basis[:, 2], left_hand_basis[:, 0])
        left_hand_basis /= np.linalg.norm(left_hand_basis, axis=0, keepdims=True)

        left_thumb_0_direction = arms_position_1["left_finger_directions"][0, 0] @ left_hand_basis.T
        v1, v2, v3 = bone_matrix.transposed() @ Vector(left_thumb_0_direction[[0, 2, 1]] * np.array([1, 1, -1]))
        if np.isclose(v3*v3, 1):
            beta = np.sign(v3) * pi/2.
            gamma = 0.
        else:
            gamma = np.arctan2(-v1, v2)
            beta = np.arctan2(v3, -v1/np.sin(gamma)) if np.isclose(v2, 0) else np.arctan2(v3, v2/np.cos(gamma))

        rotation_matrix_bone = (bone_matrix.transposed() @ rotation_matrix @ bone_matrix @
                                Matrix().Rotation(gamma, 3, 'Z') @ Matrix().Rotation(beta, 3, 'X'))
        bone.rotation_euler = rotation_matrix_bone.to_euler("YXZ")


        bone_names_ = ["r_index_0", "r_middle_0", "r_ring_0", "r_pinky_0"]

        right_forearm_direction = half_world_landmarks_[7] - half_world_landmarks_[5]
        right_forearm_direction /= np.linalg.norm(right_forearm_direction)
        if np.isclose(arms_position_["right_elbow_angle"], pi):
            right_arm_rotation = arms_position_["right_shoulder_rotation"] + arms_position_["right_elbow_rotation"] + pi/2.
            aux_vector = max_right_shoulder_rotation(arms_position_["right_shoulder_direction"]) @ right_shoulder_basis.T
            right_palm_normal_vector_max_arm_rotation = np.cross(aux_vector, right_forearm_direction)
            right_palm_normal_vector_no_wrist_inclination = (np.cos(right_arm_rotation) * right_palm_normal_vector_max_arm_rotation -
                                                           np.sin(right_arm_rotation) * aux_vector)
        else:
            right_palm_normal_vector_no_elbow_rotation = np.cross(right_forearm_direction, arms_position_["right_shoulder_direction"] @ right_shoulder_basis.T)
            right_palm_normal_vector_no_elbow_rotation /= np.linalg.norm(right_palm_normal_vector_no_elbow_rotation)
            aux_vector = np.cross(right_forearm_direction, right_palm_normal_vector_no_elbow_rotation)
            right_palm_normal_vector_no_wrist_inclination = (np.cos(arms_position_["right_elbow_rotation"]) * right_palm_normal_vector_no_elbow_rotation -
                                                             np.sin(arms_position_["right_elbow_rotation"]) * aux_vector)
        v1 = right_palm_normal_vector_no_wrist_inclination[[0, 2, 1]] * np.array([1, 1, -1])
        v3 = right_forearm_direction[[0, 2, 1]] * np.array([1, 1, -1])
        v2 = np.cross(v3, v1)
        parent_matrix = Matrix(np.array([v1, v3, -v2]).T)
        # parent_matrix = armature.pose.bones[bone_names[7]].matrix.to_3x3()

        a = np.cos(arms_position_["right_wrist_inclination"])
        b = np.sin(arms_position_["right_wrist_inclination"])
        c = np.cos(arms_position_["right_wrist_rotation"])
        d = np.sin(arms_position_["right_wrist_rotation"])
        rotation_matrix_parent_0 = Matrix([
            [1.+(a-1.)*c*c, -b*c, (1.-a)*c*d],
            [b*c, a, -b*d],
            [(1.-a)*c*d, b*d, 1.+(a-1.)*d*d]
        ])
        a = np.cos(arms_position_1["right_wrist_inclination"])
        b = np.sin(arms_position_1["right_wrist_inclination"])
        c = np.cos(arms_position_1["right_wrist_rotation"])
        d = np.sin(arms_position_1["right_wrist_rotation"])
        rotation_matrix_parent_1 = Matrix([
            [1.+(a-1.)*c*c, b*c, (1.-a)*c*d],
            [-b*c, a, b*d],
            [(1.-a)*c*d, -b*d, 1.+(a-1.)*d*d]
        ])
        rotation_matrix = parent_matrix @ rotation_matrix_parent_1 @ rotation_matrix_parent_0 @ parent_matrix.transposed()
        for bone_name in bone_names_:
            bone = armature.pose.bones[bone_name]
            bone.rotation_mode = 'YXZ'

            bone_matrix = bone.matrix.to_3x3()
            rotation_matrix_bone = bone_matrix.transposed() @ rotation_matrix @ bone_matrix
            bone.rotation_euler = rotation_matrix_bone.to_euler("YXZ")

        bone_name = "r_thumb_0"
        bone = armature.pose.bones[bone_name]
        bone.rotation_mode = 'YXZ'
        bone_matrix = bone.matrix.to_3x3()

        right_hand_basis = np.empty((3, 3), dtype=np.float64)
        right_hand_basis[:, 0] = np.cross(half_world_landmarks_[17] - half_world_landmarks_[7], half_world_landmarks_[41] - half_world_landmarks_[7])
        right_hand_basis[:, 2] = (half_world_landmarks_[41] + half_world_landmarks_[17])/2. - half_world_landmarks_[7]
        right_hand_basis[:, 1] = np.cross(right_hand_basis[:, 2], right_hand_basis[:, 0])
        right_hand_basis /= np.linalg.norm(right_hand_basis, axis=0, keepdims=True)

        right_thumb_0_direction = arms_position_1["right_finger_directions"][0, 0] @ right_hand_basis.T
        v1, v2, v3 = bone_matrix.transposed() @ Vector(right_thumb_0_direction[[0, 2, 1]] * np.array([1, 1, -1]))
        if np.isclose(v3*v3, 1):
            beta = np.sign(v3) * pi/2.
            gamma = 0.
        else:
            gamma = np.arctan2(-v1, v2)
            beta = np.arctan2(v3, -v1/np.sin(gamma)) if np.isclose(v2, 0) else np.arctan2(v3, v2/np.cos(gamma))

        rotation_matrix_bone = (bone_matrix.transposed() @ rotation_matrix @ bone_matrix @
                                Matrix().Rotation(gamma, 3, 'Z') @ Matrix().Rotation(beta, 3, 'X'))
        bone.rotation_euler = rotation_matrix_bone.to_euler("YXZ")


        # Update position
        bpy.context.view_layer.update()
        for bone_number in range(6, 50):
            bone = armature.pose.bones[bone_names[bone_number]]
            half_world_landmarks_[bone_number-2] = np.array(bone.tail, dtype=np.float64)[[0, 2, 1]] * np.array([1, -1, 1])
        arms_position_.update(landmarks2arm_position(half_world_landmarks_, 1, 1))


        # Fingers
        hand_basis = np.empty((2, 3, 3), dtype=np.float64)
        hand_basis[0, :, 0] = np.cross(half_world_landmarks_[40] - half_world_landmarks_[6], half_world_landmarks_[16] - half_world_landmarks_[6])
        hand_basis[0, :, 2] = (half_world_landmarks_[40] + half_world_landmarks_[16])/2. - half_world_landmarks_[6]
        hand_basis[0, :, 1] = np.cross(hand_basis[0, :, 2], hand_basis[0, :, 0])

        hand_basis[1, :, 0] = np.cross(half_world_landmarks_[17] - half_world_landmarks_[7], half_world_landmarks_[41] - half_world_landmarks_[7])
        hand_basis[1, :, 2] = (half_world_landmarks_[41] + half_world_landmarks_[17])/2. - half_world_landmarks_[7]
        hand_basis[1, :, 1] = np.cross(hand_basis[1, :, 2], hand_basis[1, :, 0])

        hand_basis /= np.linalg.norm(hand_basis, axis=1, keepdims=True)

        for phalanx in range(1, 4):
            for i, side in enumerate(["left", "right"]):
                bone_names_ = ["%s_%s_%d" % (side[0], finger, phalanx) for finger in ["thumb", "index", "middle", "ring", "pinky"]]

                finger_directions_image = (arms_position_1["%s_finger_directions" % side][phalanx] @ hand_basis[i].T)[:, [0, 2, 1]] * np.array([1, 1, -1])
                for finger in range(5):
                    bone = armature.pose.bones[bone_names_[finger]]
                    bone.rotation_mode = 'YXZ'
                    
                    v1, v2, v3 = bone.matrix.transposed().to_3x3() @ Vector(finger_directions_image[finger])
                    if np.isclose(v2, 1):
                        beta = 0.
                        gamma = 0.
                    else:
                        if np.isclose(v3*v3, 1):
                            beta = np.sign(v3) * pi/2.
                            gamma = 0.
                        else:
                            gamma = np.arctan2(-v1, v2)
                            beta = np.arctan2(v3, -v1/np.sin(gamma)) if np.isclose(v2, 0) else np.arctan2(v3, v2/np.cos(gamma))
                    bone.rotation_euler = (beta, 0, gamma)

            # Update position
            bpy.context.view_layer.update()
            for bone_number in range(6, 50):
                bone = armature.pose.bones[bone_names[bone_number]]
                half_world_landmarks_[bone_number-2] = np.array(bone.tail, dtype=np.float64)[[0, 2, 1]] * np.array([1, -1, 1])
            arms_position_.update(landmarks2arm_position(half_world_landmarks_, 1, 1))


    # Start animation
    bpy.ops.object.mode_set(mode='POSE')

    if isinstance(list_time_movement, (int, float)):
        list_time_movement = [list_time_movement] * (len(list_half_landmarks) - 1)

    if isinstance(list_time_position, (int, float)):
        list_time_position = [list_time_position] * len(list_half_landmarks)

    if isinstance(list_frame_dimensions, tuple):
        list_frame_dimensions = [list_frame_dimensions] * len(list_half_landmarks)

    T = 0
    for half_landmarks in list_half_landmarks:
        # Set the armature's position as half_world_landmarks
        arms_position = landmarks2arm_position(half_landmarks, *list_frame_dimensions.pop(0))
        for k, v in arms_position.items():
            if np.isnan(v).any():
                arms_position[k] = arms_position_armature[k]
        set_pose(half_landmarks_armature, arms_position_armature, arms_position)

        # Set the pose in the first frame
        for bone_name in arms_bones_names:
            bone = armature.pose.bones[bone_name]
            bone.keyframe_insert(data_path="rotation_euler", frame=int(T*fps))

        T += list_time_position.pop(0)

        # Set the pose in the last frame
        for bone_name in arms_bones_names:
            bone = armature.pose.bones[bone_name]
            bone.keyframe_insert(data_path="rotation_euler", frame=int(T*fps))

        # Clear the rotations of the armature
        bpy.ops.pose.select_all(action='SELECT')
        bpy.ops.pose.rot_clear()

        if list_time_movement:
            T += list_time_movement.pop(0)

    # End and save animation
    bpy.ops.object.mode_set(mode='OBJECT')
    bpy.ops.wm.save_as_mainfile(filepath=animation_filepath)

In [9]:
half_landmarks_0 = get_half_landmarks(total_results, "A", 27)
half_landmarks_1 = get_half_landmarks(total_results, "M", 35)

move_arms_blender_v1(
    "../Datos/Procesados/avatar_armature.blend", "Armature", half_landmarks_rest, arms_position_rest,
    [half_landmarks_rest, half_landmarks_0, half_landmarks_1, half_landmarks_rest],
    [(1, 1), (320, 240), (320, 240), (1, 1)],
    1, 0.5, 24, "prueba_dedos_2.blend"
)

Info: Saved "prueba_dedos_2.blend"


In [10]:
letters = ["A", "M"]
half_landmarks_letters = np.empty((2, 48, 3), dtype=np.float64)

for i, letter in enumerate(letters):
    bpy.ops.wm.open_mainfile(filepath="avatar_%s.blend" % letter)
    armature = bpy.data.objects.get("Armature")
    bpy.context.view_layer.objects.active = armature
    
    bpy.ops.object.mode_set(mode='POSE')
    
    half_landmarks_letters[i, :4] = trunk_landmarks_avatar.copy()
    
    for bone_number in range(6, 50):
        bone = armature.pose.bones[bone_names[bone_number]]
        half_landmarks_letters[i, bone_number-2] = np.array(bone.tail, dtype=np.float64)[[0, 2, 1]] * np.array([1, -1, 1])

In [11]:
move_arms_blender_v1(
    "../Datos/Procesados/avatar_armature.blend", "Armature", half_landmarks_rest, arms_position_rest,
    [half_landmarks_rest, half_landmarks_letters[0], half_landmarks_letters[1], half_landmarks_rest],
    (1, 1), 1, 0.5, 24, "prueba_dedos_1.blend"
)

Info: Saved "prueba_dedos_1.blend"
